# Step 3: Creating the model inference script
To submit your algorithm to the challenge, you need to create an inference docker container. 

In [1]:
import os

print(f"Previous exec path: {os.getcwd()}")
# move two level up
os.chdir("../../")
print(f"Current exec path: {os.getcwd()}")

import json
import torch
import creationism
from tqdm import tqdm

from wholeslidedata.interoperability.asap.annotationwriter import write_point_set
from wholeslidedata.image.wholeslideimage import WholeSlideImage
from wholeslidedata.iterators import create_patch_iterator, PatchConfiguration
from wholeslidedata.annotation.labels import Label

from source.utils.wsdetectron2 import Detectron2DetectionPredictor
from source.utils.structures import Point

Previous exec path: /workspace/source/notebooks
Current exec path: /workspace


In [2]:
# set up path one level above
os.chdir("..")
print(os.getcwd())

/workspace


Setting up the paths.

In [3]:
image_path = r"./data/monkey-data/images/pas-cpg/A_P000002_PAS_CPG.tif"
mask_path = r"./data/monkey-data/images/tissue-masks/A_P000002_mask.tif"
output_path = r"./outputs/results"
if not (os.path.isdir(output_path)):
    os.mkdir(output_path)
json_filename_immune_cells = "detected-inflammatory-cells.json"
json_filename_lymphocytes = "detected-lymphocytes.json"
json_filename_monocytes = "detected-monocytes.json"

print(f"Pytorch GPU available: {torch.cuda.is_available()}")
print(image_path, mask_path)

Pytorch GPU available: True
./data/monkey-data/images/pas-cpg/A_P000002_PAS_CPG.tif ./data/monkey-data/images/tissue-masks/A_P000002_mask.tif


Defining patch configuration for each image.

In [4]:
patch_shape = (128, 128, 3)
spacings = (0.5,)
overlap = (0, 0)
offset = (0, 0)
center = False

patch_configuration = PatchConfiguration(
    patch_shape=patch_shape,
    spacings=spacings,
    overlap=overlap,
    offset=offset,
    center=center,
)

Loading the saved model.

In [5]:
model = Detectron2DetectionPredictor(
    output_dir=output_path,
    threshold=0.1,
    nms_threshold=0.2,
    weight_root="./outputs/model_final.pth",
)

Creating a patch iterator using the roi mask and sliding windows.

In [6]:
iterator = create_patch_iterator(
    image_path=image_path,
    mask_path=mask_path,
    patch_configuration=patch_configuration,
    cpus=4,
    backend="asap",
)  # was backend='asap'

Some useful functions.

In [7]:
def px_to_mm(px: int, spacing: float):
    return px * spacing / 1000


def to_wsd(points, label: str = "lymphocyte"):
    if label == "lymphocyte":
        label = Label("lymphocyte", 1, color="red")
    elif label == "monocyte":
        label = Label("monocyte", 2, color="green")
    else:
        label = Label("inflammatory-cell", 3, color="blue")
    """Convert list of coordinates into WSD points"""
    new_points = []
    for i, point in enumerate(points):
        p = Point(
            index=i,
            label=label,
            coordinates=[point],
        )
        new_points.append(p)
    return new_points


def write_json_file(*, location, content):
    # Writes a json file
    with open(location, "w") as f:
        f.write(json.dumps(content, indent=4))

Run inference on an image with loaded model.

In [9]:
def inference(iterator, predictor, spacing, image_path, output_path):
    from copy import deepcopy

    print("predicting...")

    SPACING_CONST = 0.24199951445730394

    json_filename_immune_cells = "detected-inflammatory-cells.json"
    json_filename_lymphocytes = "detected-lymphocytes.json"
    json_filename_monocytes = "detected-monocytes.json"

    output_dict = {
        "name": "",
        "type": "Multiple points",
        "version": {"major": 1, "minor": 0},
        "points": [],
    }

    output_dict_immune_cells = deepcopy(output_dict)
    output_dict_lymphocytes = deepcopy(output_dict)
    output_dict_monocytes = deepcopy(output_dict)

    output_dict_immune_cells["name"] = "inflammatory-cells"
    output_dict_lymphocytes["name"] = "lymphocytes"
    output_dict_monocytes["name"] = "monocytes"

    annotations_immune_cells = []
    annotations_lymphocytes = []
    annotations_monocytes = []

    counter_immune_cells = 0
    counter_lymphocytes = 0
    counter_monocytes = 0

    # NOTE / TODO: i used a different spacing for the image (0.24199951445730394), so we need to be shure how this works...
    spacing_min = (
        0.25  # was used in the original code to edit the annotations to bounding boxes
    )
    ratio = spacing / spacing_min
    with WholeSlideImage(image_path) as wsi:
        spacing = wsi.get_real_spacing(spacing_min)
        print(f"Spacing: {spacing} - Spacing const: {SPACING_CONST} - ratio: {ratio}")

    for x_batch, y_batch, info in tqdm(iterator):
        x_batch = x_batch.squeeze(0)
        y_batch = y_batch.squeeze(0)

        predictions = predictor.predict_on_batch(x_batch)
        for idx, prediction in enumerate(predictions):
            c = info["x"]
            r = info["y"]

            for detections in prediction:
                x, y, label, confidence = detections.values()
                # print(f"Detected {label} at {x}, {y} with confidence {confidence}")

                if x == 128 or y == 128:
                    continue

                if y_batch[idx][y][x] == 0:
                    continue

                x = x * ratio + c  # x is in spacing = 0.5 but c is in spacing = 0.25
                y = y * ratio + r

                prediction_record_immune_cells = {
                    "name": "Point " + str(counter_immune_cells),
                    "point": [
                        px_to_mm(x, spacing),
                        px_to_mm(y, spacing),
                        SPACING_CONST,
                    ],
                    "probability": confidence,
                }
                output_dict_immune_cells["points"].append(
                    prediction_record_immune_cells
                )
                annotations_immune_cells.append((x, y))
                counter_immune_cells += 1

                if label == "lymphocyte":  # lymphocyte
                    prediction_record_lymphocytes = {
                        "name": "Point " + str(counter_lymphocytes),
                        "point": [
                            px_to_mm(x, spacing),
                            px_to_mm(y, spacing),
                            SPACING_CONST,
                        ],
                        "probability": confidence,
                    }
                    output_dict_lymphocytes["points"].append(
                        prediction_record_lymphocytes
                    )
                    annotations_lymphocytes.append((x, y))
                    counter_lymphocytes += 1

                elif label == "monocyte":  # monocyte
                    prediction_record_monocytes = {
                        "name": "Point " + str(counter_monocytes),
                        "point": [
                            px_to_mm(x, spacing),
                            px_to_mm(y, spacing),
                            SPACING_CONST,
                        ],
                        "probability": confidence,
                    }
                    output_dict_monocytes["points"].append(prediction_record_monocytes)
                    annotations_monocytes.append((x, y))
                    counter_monocytes += 1

                else:
                    print("Unknown label")
                    continue

    print(f"Predicted {len(annotations_immune_cells)} points")
    print("saving predictions...")

    # for i, points in enumerate(annotations):
    #     print(f"Annotation {i}: {points}")

    # saving json file immune cells
    output_path_json_immune_cells = os.path.join(
        output_path, json_filename_immune_cells
    )
    write_json_file(
        location=output_path_json_immune_cells, content=output_dict_immune_cells
    )

    # saving json file lymphocytes
    output_path_json_lyphocytes = os.path.join(output_path, json_filename_lymphocytes)
    write_json_file(
        location=output_path_json_lyphocytes, content=output_dict_lymphocytes
    )

    # saving json file monocytes
    output_path_json_monocytes = os.path.join(output_path, json_filename_monocytes)
    write_json_file(location=output_path_json_monocytes, content=output_dict_monocytes)

    # #TODO: bugged code, they had the same problem and even downgrading shapely didn't work :(
    # # saving xml file
    # annotations_wsd = to_wsd(annotations_immune_cells, label="inflammatory-cell")
    # xml_filename = 'points_results.xml'
    # output_path_xml = os.path.join(output_path,xml_filename)
    # write_point_set(
    #     annotations_wsd,
    #     output_path_xml,
    #     label_color="blue",
    # )

    print("finished!")

In [10]:
inference(
    iterator=iterator,
    predictor=model,
    spacing=spacings[0],
    image_path=image_path,
    output_path=output_path,
)

iterator.stop()

predicting...
Spacing: 0.24199951445730394 - Spacing const: 0.24199951445730394 - ratio: 2.0


  0%|          | 0/379 [00:00<?, ?it/s]

/venv/lib/python3.8/site-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride
/venv/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|██████████| 379/379 [00:42<00:00,  8.95it/s]


Predicted 14925 points
saving predictions...
finished!
